In [ ]:
!pip install transformers datasets accelerate pandas numpy evaluate mlflow

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset
from google.colab import drive
import evaluate

def parse_emotion_file(file_path):
    """
    Parses a text file with each line in the format: {text; emotion}
    and returns a pandas DataFrame with 'text' and 'emotion' columns.

    Args:
    - file_path (str): Path to the .txt file to be parsed

    Returns:
    - df (pd.DataFrame): DataFrame containing 'text' and 'emotion' columns
    """
    texts = []
    emotions = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Split each line by the semicolon separator
                text, emotion = line.strip().split(';')

                # append text and emotion to separate lists
                texts.append(text)
                emotions.append(emotion)
            except ValueError:
                continue

    return pd.DataFrame({'text': texts, 'emotion': emotions})


In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


GPU available: True
GPU device: Tesla T4
GPU memory: 14.74 GB


In [ ]:
## Connecting to the Google Drive
from google.colab import drive
drive.mount('/content/drive')

PARENT_DIR = '/content/drive/MyDrive/MLEng/'
MODEL_DIR = os.path.join(PARENT_DIR, 'model_outputs/')
os.makedirs(MODEL_DIR, exist_ok=True)

# Update file paths
# Parse text files and store as Pandas DataFrames
train_df = parse_emotion_file(PARENT_DIR + "data/train.txt")
val_df = parse_emotion_file(PARENT_DIR + "data/val.txt")
test_df = parse_emotion_file(PARENT_DIR + "data/test.txt")

Mounted at /content/drive


In [ ]:
print(f"Train set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")
print(f"Emotion classes: {train_df['emotion'].unique()}")
print(f"Class distribution in training set:")
print(train_df['emotion'].value_counts())

Train set: 16000 samples
Validation set: 2000 samples
Test set: 2000 samples
Emotion classes: ['sadness' 'anger' 'love' 'surprise' 'fear' 'joy']
Class distribution in training set:
emotion
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64


In [ ]:
# Define the model path for DistilBERT
model_name = "distilbert-base-uncased"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create label mappings
labels = sorted(train_df["emotion"].unique())
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

print(f"Label to ID mapping: {label2id}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Label to ID mapping: {'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}


In [ ]:
# Convert pandas DataFrames to HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Define tokenization function with batching for efficiency
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128  # Reduced from 512 for better performance
    )

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Add labels
def add_labels(examples):
    examples["labels"] = [label2id[emotion] for emotion in examples["emotion"]]
    return examples

train_dataset = train_dataset.map(add_labels, batched=True)
val_dataset = val_dataset.map(add_labels, batched=True)
test_dataset = test_dataset.map(add_labels, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label
)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
# Set up training arguments optimized for T4 GPU
training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_DIR, "checkpoints"),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,  # Increased batch size
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    save_total_limit=2,  # Keep only the 2 best checkpoints to save space
    fp16=True,  # Enable mixed precision training for faster performance on T4
    gradient_accumulation_steps=2  # Accumulate gradients for effective larger batch size
)

# Initialize trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-ed55649eae2f>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


MLFlow Tracking

In [ ]:
import mlflow
from mlflow.models import infer_signature

mlflow.set_tracking_uri("sqlite:///mlflow.db") #("http://localhost:5000")
# mlflow.set_experiment("emotion_classification_experiment")

with mlflow.start_run(run_name = "distilbert_emotion_classifier"): # as run:
  # Log Params
  mlflow.log_param("model_name", model_name)
  mlflow.log_param("max_length", 512)
  mlflow.log_param("batch_size", training_args.per_device_train_batch_size)
  mlflow.log_param("learning_rate", training_args.learning_rate)
  mlflow.log_param("epochs", training_args.num_train_epochs)

  # Train the model
  trainer.train()

  # Evaluate and log metrics
  eval_results = trainer.evaluate()
  mlflow.log_metrics({
      "eval_loss": eval_results["eval_loss"],
      "eval_accuracy": eval_results["eval_accuracy"]
  })

  # Log the model
  mlflow.transformers.log_model(
      transformers_model = {
          "model": model,
          "tokenizer": tokenizer
      },
      artifact_path = "distilbert-emotion-classifier",
      task = "text-classification",
      signature = infer_signature(
          train_dataset[:2]["input_ids"],
          np.array([id2label[i] for i in train_dataset[:2]["labels"].numpy()])
      )
  )

2025/03/30 17:20:26 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/03/30 17:20:27 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: afrologicinsect to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.209734,0.923500
2,0.395300,0.150498,0.936000
3,0.395300,0.140088,0.937000
4,0.101300,0.138142,0.940500
5,0.101300,0.134733,0.939500


2025/03/30 17:25:08 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Device set to use cuda:0


README.md:   0%|          | 0.00/8.58k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

2025/03/30 17:25:10 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/03/30 17:25:10 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/03/30 17:25:31 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local

wandb pass: ""

In [ ]:
# Evaluate the model on the test set
test_results = trainer.evaluate(test_dataset)
print(f"Test results: {test_results}")

# Save the final model to the parent directory
final_model_path = os.path.join(MODEL_DIR, "final_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"Model saved to {final_model_path}")

Test results: {'eval_loss': 0.1534700095653534, 'eval_accuracy': 0.925, 'eval_runtime': 1.7026, 'eval_samples_per_second': 1174.707, 'eval_steps_per_second': 18.795, 'epoch': 5.0}
Model saved to /content/drive/MyDrive/MLEng/model_outputs/final_model


In [ ]:
# Save label mappings for inference
import json
with open(os.path.join(final_model_path, "label_mappings.json"), "w") as f:
    json.dump({"label2id": label2id, "id2label": id2label}, f)

# Example of how to use the model for inference
from transformers import pipeline

emotion_classifier = pipeline(
    "text-classification",
    model=final_model_path,
    tokenizer=final_model_path
)

# Test inference with a sample text
test_text = "I'm feeling very happy today!"
result = emotion_classifier(test_text)
print(f"Sample text: '{test_text}'")
print(f"Predicted emotion: {result[0]['label']}, Score: {result[0]['score']:.4f}")

Device set to use cuda:0


Sample text: 'I'm feeling very happy today!'
Predicted emotion: joy, Score: 0.9988


In [ ]:
# Save model, tokenizer and label mappings
output_dir = PARENT_DIR + "model_export/"
os.makedirs(output_dir, exist_ok=True)

# Save the model
trainer.save_model(output_dir)

# Save the tokenizer (you already have these files)
tokenizer.save_pretrained(output_dir)

# Save label mappings (you already have this file)
with open(os.path.join(output_dir, 'label_mappings.json'), 'w') as f:
    json.dump({
        'label2id': label2id,
        'id2label': id2label
    }, f)

# Check that the model weights file exists
import os
print("Files in model directory:")
print(os.listdir(output_dir))

Files in model directory:
['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json', 'training_args.bin', 'label_mappings.json']
